In [123]:
import os
from openai import OpenAI
import openai
from PIL import Image
import base64
from io import BytesIO
import requests

In [124]:
# os.environ["OPENAI_API_KEY"] = "sk-6OyTxsQcuf5tBz4w221b5822EfA143539f03A7F7BaB778D4"
os.environ["OPENAI_API_KEY"] = "sk-Z3eLGZ6uA0AgDKwz88C0D9E17c4347Cb845cE6C85c8fD2Ee"
os.environ["OPENAI_BASE_URL"] = "https://api.xiaoai.plus/v1"

client = OpenAI()

GPT_MODEL = "gpt-4o"
DALLE = "dall-e-3"

# prompt = "give the brief description of image"
prompt = "give the description of what to imaging in a line"
# prompt = "Generates prompt that is inspirational and suggestive, it can be used to recreate an uploaded image exactly"
# prompt = "Generates prompt briefly that is inspirational and suggestive, it can be used to recreate an uploaded image exactly"
# prompt = "Carefully analyze the provided image and generate an exhaustive description that captures every element essential for its recreation. Detail the spatial arrangement, key subjects, and any background elements. Specify the color palette, noting any gradients or unique shades present. Describe any actions or interactions between subjects, and capture the mood or atmosphere effectively. Your description should be precise and comprehensive, enabling an artist or a generative model to replicate the image with high fidelity based on your text alone."

In [125]:
# Encode the image at the given path to a base64 string
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Get the base64 encoded description of an image at the given path
def get_image_description(image_path):
    encoded_image = encode_image(image_path)
    return encoded_image

# Generate a detailed description of the image using GPT-4
def generate_description_with_gpt(base64_image, prompt):
    response = client.chat.completions.create(
        model=GPT_MODEL,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt,
                    },
                    {
                        "type": "image_url",
                        "image_url": {
                            "url": f"data:image/png;base64,{base64_image}"
                        }
                    },
                ]
            }
        ],
        stream=False,
        max_tokens=300,
    )
    summary = response.choices[0].message.content.strip()
    return summary 

# Process a list of image paths and generate descriptions for each
def process_images(image_paths, prompt_path):
    descriptions = []
    for image_path in image_paths:
        image_description = get_image_description(image_path)
        description = generate_description_with_gpt(image_description, prompt)
        descriptions.append(description)
    return descriptions

In [126]:
def get_images_path(set_path):

    file_paths = []

    for root, _, files in os.walk(set_path):
        for file in files:
            file_paths.append(os.path.join(root, file))

    return file_paths

def write_prompt_out(descriptions, prompt_path):
    with open(prompt_path, "w") as file:
        for line in descriptions:
            file.write(line + "\n")


In [127]:
def generate_image(prompt):
    response = openai.images.generate(
        model=DALLE,
        prompt=prompt,
        n=1,
        size="1024x1024"
    )
    # summary = response.choices[0].message.content.strip()
    # image_url = response['data'][0]['url']
    image_url = response.data[0].url
    return image_url

def download_image(url, save_path):
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    img.save(save_path)
    print(f"Image saved to {save_path}")

In [128]:
def is_folder_exist(save_path):
    """
    Check if a directory exists, and create it if it doesn't.

    Args:
    path (str): The path to the directory.
    """
    if not os.path.exists(save_path):
        os.makedirs(save_path)
        print(f"Directory '{save_path}' created.")
    else:
        print(f"Directory '{save_path}' already exists.")

    return save_path

def read_file_to_list(file_path):
    """
    Read a text file and return a list where each element is a line from the file.

    Args:
    file_path (str): The path to the text file.

    Returns:
    list: A list of lines from the text file.
    """
    lines = []
    with open(file_path, 'r', encoding='utf-8') as file:
        lines = file.readlines()
    
    # Remove any trailing newline characters
    lines = [line.strip() for line in lines]
    return lines

In [129]:
if __name__ == "__main__":
    set_path = "./testset"
    save_folder = "./output"
    output_folder = is_folder_exist(save_folder + '/image')
    prompt_path = os.path.join(save_folder, 'prompt.txt')
    image_paths = get_images_path(set_path)
    descriptions = process_images(image_paths, prompt_path)
    
    write_prompt_out(descriptions, prompt_path)
    for i, description in enumerate(descriptions):
        print(f"Description for image {i+1}: {description}")
        prompt = "generate image by the description: " + description
        image_url = generate_image(prompt)
        save_path = os.path.join(output_folder, str(i) + '.png')
        download_image(image_url, save_path)

Directory './output/image' already exists.
Description for image 1: The image depicts a group of people seated together in what appears to be a social setting, possibly a bar or a lounge. One person is holding a bottle wrapped in a clear plastic bag, while another person is holding a colorful cylindrical object. There are drinks and straws on the table in front of them, suggesting they are enjoying a beverage. The background shows some dim lighting and possibly some decorative elements on the wall.
Image saved to ./output/0.png
Description for image 2: The image appears to be of a formal event, possibly a wedding or banquet. In the foreground, a person in a gray suit with a boutonniere and another individual are seated at a table with red chairs. Glasses of beverages are on the table. In the background, people in formal or semi-formal attire, including servers in red vests, are visible, indicating a celebratory or upscale atmosphere.
Image saved to ./output/1.png
Description for image 